In [2]:
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'myyieldbucket' # Mention the created S3 bucket name here
print("Using bucket " + bucket)


Using bucket myyieldbucket


In [3]:
df = pd.read_excel("last.xlsx")

In [4]:
df.head()

,Rainfall(mm),Minimum Temparature(C),Maximum Temperature (C),Ph,N(Kg/Hectar),P(Kg/Hectar),K(Kg/Hectar),Zn(ppm),Fe(ppm),Cu(ppm),...,District_Warangal,District_Warangal urban,District_Yadadri,Season_Kharif,Season_rabi,Crop_Groundnut,Crop_Maize,Crop_Moong(Green Gram),Crop_Rice,Crop_cotton(lint)
0,156.04,29.10,40.50,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
1,165.74,26.10,37.60,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
2,240.52,27.30,38.30,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
3,180.18,27.60,37.40,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
4,256.06,25.38,39.94,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1


In [5]:
features = list(df.columns)
features

['Rainfall(mm)',
 'Minimum Temparature(C)',
 'Maximum Temperature (C)',
 'Ph',
 'N(Kg/Hectar)',
 'P(Kg/Hectar)',
 'K(Kg/Hectar)',
 'Zn(ppm)',
 'Fe(ppm)',
 'Cu(ppm)',
 'Mn(ppm)',
 'Irrigation(MBGL)',
 'Yield (Bales/Hectare)',
 'District_Adilabad',
 'District_Bhadradri',
 'District_Jagitial',
 'District_Jangoan',
 'District_Jayashankar',
 'District_Jogulamba',
 'District_Kamareddy',
 'District_Karimnagar',
 'District_Khammam',
 'District_Komaram bheem asifabad',
 'District_Mahabubabad',
 'District_Mahbubnagar',
 'District_Mancherial',
 'District_Medak',
 'District_Medchal',
 'District_Mulugu',
 'District_Nagarkurnool',
 'District_Nalgonda',
 'District_Narayanapet',
 'District_Nirmal',
 'District_Nizamabad',
 'District_Peddapalli',
 'District_Rajanna',
 'District_Rangareddy',
 'District_Sangareddy',
 'District_Siddipet',
 'District_Suryapet',
 'District_Vikarabad',
 'District_Wanaparthy',
 'District_Warangal',
 'District_Warangal urban',
 'District_Yadadri',
 'Season_Kharif',
 'Season_rab

In [6]:
features[12]

'Yield (Bales/Hectare)'

In [7]:
label = features.pop(12)
label

'Yield (Bales/Hectare)'

In [8]:
x = df[features]
y = df[label]

In [9]:
x.head()

,Rainfall(mm),Minimum Temparature(C),Maximum Temperature (C),Ph,N(Kg/Hectar),P(Kg/Hectar),K(Kg/Hectar),Zn(ppm),Fe(ppm),Cu(ppm),...,District_Warangal,District_Warangal urban,District_Yadadri,Season_Kharif,Season_rabi,Crop_Groundnut,Crop_Maize,Crop_Moong(Green Gram),Crop_Rice,Crop_cotton(lint)
0,156.04,29.10,40.50,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
1,165.74,26.10,37.60,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
2,240.52,27.30,38.30,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
3,180.18,27.60,37.40,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1
4,256.06,25.38,39.94,7.55,180.0,17.6,284.0,1.35,14.0,0.58,...,0,0,0,1,0,0,0,0,0,1


In [10]:
y.head()

0    1.88
1    2.49
2    2.72
3    3.15
4    2.55
Name: Yield (Bales/Hectare), dtype: float64

In [11]:
x.shape

(1136, 51)

In [12]:
y.value_counts()

0.95    29
0.64    20
0.90    20
2.07    15
0.79    14
        ..
1.66     1
1.49     1
2.69     1
1.46     1
3.84     1
Name: Yield (Bales/Hectare), Length: 451, dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=19)

In [14]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(908, 51)
(228, 51)
(908,)
(228,)


In [15]:
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)
testX[label] = y_test

In [16]:
print(trainX.shape)
print(testX.shape)

(908, 52)
(228, 52)


In [17]:
trainX.head()

,Rainfall(mm),Minimum Temparature(C),Maximum Temperature (C),Ph,N(Kg/Hectar),P(Kg/Hectar),K(Kg/Hectar),Zn(ppm),Fe(ppm),Cu(ppm),...,District_Warangal urban,District_Yadadri,Season_Kharif,Season_rabi,Crop_Groundnut,Crop_Maize,Crop_Moong(Green Gram),Crop_Rice,Crop_cotton(lint),Yield (Bales/Hectare)
144,240.52,27.300000,38.300000,7.55,180.0,17.60,284.00,1.35,14.00,0.58,...,0,0,1,0,0,1,0,0,0,3.46
519,255.82,17.600000,46.000000,7.52,252.0,21.30,323.30,0.90,11.27,2.68,...,0,0,1,0,0,0,0,1,0,4.15
762,9.70,30.000000,41.900000,7.50,97.0,35.02,268.00,0.75,9.80,0.71,...,0,0,0,1,0,1,0,0,0,6.81
100,129.30,23.933333,39.266667,7.53,290.3,50.25,256.15,1.45,7.90,3.05,...,0,0,1,0,0,0,0,0,1,2.69
73,106.62,26.500000,36.000000,7.30,184.6,16.90,232.90,1.10,13.02,1.36,...,0,0,1,0,0,0,0,0,1,1.53


In [18]:
trainX.to_csv("train-V-1.csv",index = False)
testX.to_csv("test-V-1.csv", index = False)

In [19]:
sk_prefix = "sagemaker/last/sklearncontainer"
trainpath = sess.upload_data(
    path="train-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test-V-1.csv", bucket=bucket, key_prefix=sk_prefix
)

In [20]:
testpath

's3://myyieldbucket/sagemaker/last/sklearncontainer/test-V-1.csv'

In [21]:
trainpath

's3://myyieldbucket/sagemaker/last/sklearncontainer/train-V-1.csv'

In [22]:
%%writefile script.py


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd


def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(input_data, model):
    if type(input_data) != str:
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=42)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    # features = list(train_df.drop('Yield (Bales/Hectare)',axis=1).values)
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestRegressor(n_estimators=args.n_estimators, random_state=args.random_state)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = r2_score(y_test,y_pred_test)
    test_rep = mean_squared_error(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing script.py


In [23]:
! python script.py --n_estimators 100 \
                   --random_state 19 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

[INFO] Extracting arguments
SKLearn Version:  0.22.1
Joblib Version:  1.2.0
[INFO] Reading data

Building training and testing datasets

Column order: 
['Rainfall(mm)', 'Minimum Temparature(C)', 'Maximum Temperature (C)', 'Ph', 'N(Kg/Hectar)', 'P(Kg/Hectar)', 'K(Kg/Hectar)', 'Zn(ppm)', 'Fe(ppm)', 'Cu(ppm)', 'Mn(ppm)', 'Irrigation(MBGL)', 'District_Adilabad', 'District_Bhadradri', 'District_Jagitial', 'District_Jangoan', 'District_Jayashankar', 'District_Jogulamba', 'District_Kamareddy', 'District_Karimnagar', 'District_Khammam', 'District_Komaram bheem asifabad', 'District_Mahabubabad', 'District_Mahbubnagar', 'District_Mancherial', 'District_Medak', 'District_Medchal', 'District_Mulugu', 'District_Nagarkurnool', 'District_Nalgonda', 'District_Narayanapet', 'District_Nirmal', 'District_Nizamabad', 'District_Peddapalli', 'District_Rajanna', 'District_Rangareddy', 'District_Sangareddy', 'District_Siddipet', 'District_Suryapet', 'District_Vikarabad', 'District_Wanaparthy', 'District_Waran

In [24]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 42,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [25]:
# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2023-07-13-09-23-58-584


Using provided s3_resource
2023-07-13 09:23:58 Starting - Starting the training job...
2023-07-13 09:24:14 Starting - Preparing the instances for training......
2023-07-13 09:25:03 Downloading - Downloading input data...
2023-07-13 09:25:34 Training - Downloading the training image...
2023-07-13 09:26:19 Uploading - Uploading generated training model2023-07-13 09:26:12,145 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-07-13 09:26:12,149 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-13 09:26:12,189 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-07-13 09:26:12,340 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-13 09:26:12,353 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-13 09:26:12,367 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-1

In [26]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2023-07-13 09:26:30 Starting - Preparing the instances for training
2023-07-13 09:26:30 Downloading - Downloading input data
2023-07-13 09:26:30 Training - Training image download completed. Training in progress.
2023-07-13 09:26:30 Uploading - Uploading generated training model
2023-07-13 09:26:30 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-south-1-109204713374/RF-custom-sklearn-2023-07-13-09-23-58-584/output/model.tar.gz


In [27]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [28]:
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2023-07-13-09-27-19


INFO:sagemaker:Creating model with name: Custom-sklearn-model-2023-07-13-09-27-19
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model-2023-07-13-09-27-19
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model-2023-07-13-09-27-19


-----!

In [29]:
testX[features][0:2].values.tolist()

[[9.585714286,
  10.6,
  46.0,
  7.87,
  301.05,
  80.77,
  490.94,
  0.3,
  8.2,
  2.13,
  11.2,
  5.32,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0],
 [91.14,
  24.06666666666667,
  38.46666666666667,
  7.55,
  243.15,
  38.06,
  265.49,
  1.733,
  4.3,
  1.91,
  11.98,
  12.63,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0]]

In [30]:
print(predictor.predict(testX[features][8:10].values.tolist()))

Please use Content-Type = 'text/csv' and, send the request!!
